
# **Imporing Package**

In [ ]:
!pip install transformers
!pip install wget
!pip install Barbar

In [ ]:
import torch
import os.path
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertModel,BertPreTrainedModel
from torch.utils.data import  random_split 
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler,Dataset
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score,matthews_corrcoef
# from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime
import random
import wget
import os
import re
import string
import sys
import shutil
from barbar import Bar
seed_val = 96

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# % matplotlib inline

In [ ]:
# data_path = './drive/My Drive/Colab/NLP/Nadi Shared Task/Data/'
data_path = 'Data/'
models_path = 'Models/'
if not os.path.exists(data_path):    
    os.makedirs(data_path)
if not os.path.exists(models_path):    
    os.makedirs(models_path)

In [ ]:
'''

put your path to folder contains three files 
train_labeled.tsv
dev_labeled.tsv
unlabeled_data_10M.tsv

this three files you could get them:
1- share them from next google drive link and put them in your drive folder then put this folder path in data_path
https://drive.google.com/open?id=1MUVnDWeozyb-pF3zxAF7kbtVkJviq1bc
2- you can download them locally and put their folder path in data_path

'''
if not os.path.isfile(data_path + 'dev_labeled.tsv'):
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1MgiF1_nmk3epPmTx8WX3L-o5L8LJ_Wx1' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1MgiF1_nmk3epPmTx8WX3L-o5L8LJ_Wx1" -O dev_labeled.tsv && rm -rf /tmp/cookies.txt
    shutil.move("dev_labeled.tsv",data_path)
    
if not os.path.isfile(data_path + 'train_labeled.tsv'):
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1m6uJQrvG1VAIRMmTgxQ5NgAUqVbUCzb6' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1m6uJQrvG1VAIRMmTgxQ5NgAUqVbUCzb6" -O train_labeled.tsv && rm -rf /tmp/cookies.txt
    shutil.move("train_labeled.tsv",data_path)
if not os.path.isfile(data_path + 'unlabeled_data_10M.tsv'):
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1BCE5ctfi7sF9h8tuK5iVwqOQLAfurbXD' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1BCE5ctfi7sF9h8tuK5iVwqOQLAfurbXD" -O unlabeled_data_10M.tsv && rm -rf /tmp/cookies.txt
    shutil.move("unlabeled_data_10M.tsv",data_path)

# Use GPU if exists



In [ ]:
#re
if torch.cuda.is_available():      
    device = torch.device("cuda")
    print("GPU : ", torch.cuda.get_device_name(0))
else:
    print("Using CPU")
    device = torch.device("cpu")

GPU :  GeForce RTX 2060


# Upload Data from drive

In [ ]:
'''
drive.mount('/content/drive')
!unzip  "/content/drive/My Drive/Colab/NLP/Nadi Shared Task/Data/NADI-2020_release_1.0.zip"
'''

'\ndrive.mount(\'/content/drive\')\n!unzip  "/content/drive/My Drive/Colab/NLP/Nadi Shared Task/Data/NADI-2020_release_1.0.zip"\n'

# Read Data using pandas

Labels Vocabulary

In [ ]:
labels_vocab = ['Iraq', 'Egypt', 'Morocco', 'Libya', 'United_Arab_Emirates', 'Mauritania', 'Saudi_Arabia', 'Bahrain', 'Syria', 'Djibouti', 'Lebanon', 'Oman', 'Palestine', 'Algeria', 'Somalia', 'Jordan', 'Tunisia', 'Kuwait', 'Yemen', 'Sudan', 'Qatar']

Load Training Data

In [ ]:
df_train = pd.read_csv(data_path + "train_labeled.tsv", delimiter='\t',header=0, names=['tweet_ID', 'tweet_content', 'country_label', 'province_label'])
df_train = df_train.drop(['tweet_ID', 'province_label'], axis=1)
print('Number of training sentences: {:,}\n'.format(df_train.shape[0]))
df_train.head(10)

Number of training sentences: 21,000



,tweet_content,country_label
0,الفار العور يشوف فقط كيسي ومايشوف ماتويد,Iraq
1,ي دينيييي ربنا يستر,Egypt
2,أساساً نسبكم قذر ونجس بلاش تتفاخروا بنجاستكم ي...,Iraq
3,ليْسَت كُل المَشَاعِرِ تَحْتَاجُ إلى حَبِيب بَ...,Morocco
4,لأ ني حاضرها هذي لايف,Libya
5,اللهم اشفي كل شخص عباله الدنيا واقفه عليه,United_Arab_Emirates
6,هيتقال عليك محروق ومش عارف تعمل زيهم :D,Egypt
7,والهوسات الزينه ابيت ماجد ياسين حمدالله ع سلام...,Iraq
8,انا من الناس المتخلفة اللي بتعمل كده للأسف بس ...,Egypt
9,مشكلة موريتان عنها ماتلينا نكد نصنفوها من الدو...,Mauritania


Load Testing Data

In [ ]:
df_test = pd.read_csv(data_path+"dev_labeled.tsv", delimiter='\t',header=0, names=['tweet_ID', 'tweet_content', 'country_label', 'province_label'])
df_test = df_test.drop(['tweet_ID', 'province_label'], axis=1)
print('Number of training sentences: {:,}\n'.format(df_test.shape[0]))
df_test.head(10)

Number of training sentences: 4,957



,tweet_content,country_label
0,@Osama__zahrani ايسكو لاعب اليوم :) اسيست وهدف,Iraq
1,بعد صلاه الفجر بقا,Egypt
2,إن شاء الله هذه المرة يكون من نصيبي,Algeria
3,ههههههههههههههههه خلي السوداني يزغبك,Yemen
4,كل حاجة محسوبة يا جماعة والله,Egypt
5,@kasimf اللهم ضيق عليه أنفاسه وعسر ميتته إلى أ...,Saudi_Arabia
6,يعني ما يحتاج نلبس ثقيل,Syria
7,(ولئن سألتهم من خلقهم ليقولن الله فأنى يؤفكون)...,Egypt
8,آستودعك يا ربي شخصاً ينبض في قلبي كل مره ، ولا...,United_Arab_Emirates
9,قُل لَّن يُصِيبَنَا إِلَّا مَا كَتَبَ اللَّهُ ...,Iraq


Load Unlabeled Data

In [ ]:
df_unlabeled = pd.read_csv(data_path+"unlabeled_data_10M.tsv", delimiter='\t',header=0, names=['tweet_ID', 'tweet_content'])

df_unlabeled = df_unlabeled.drop(['tweet_ID'], axis=1)
print('Number of training sentences: {:,}\n'.format(df_unlabeled.shape[0]))
df_unlabeled.head(10)

Number of training sentences: 9,549,500



,tweet_content
0,RT @Almajlliss: وزير التربية والنائب د.محمد ال...
1,الأسباب الخمسة للنجاح : التركيز، التميز، التنظ...
2,RT @mn9oorr_77: ⠀ ⠀⠀⠀⠀⠀⠀ تـُغار مـِن مِـينَ،وأ...
3,"RT @Mohamed__Eissa: قلبي يدق ""كصرخات طالب"" و ق..."
4,بعض الغياب يعلمك كيف تشتاق ، وبعض الغياب يعلمك...
5,اوه نسيت اقول امسينا وامسى الملك لله 😊
6,RT @b_p_QQQ: هذي هي الدنيا تجارب على الدوم تعط...
7,RT @M__I__K__A1: عن اللحظه الحلوه اللى بتغمض ف...
8,4-فالإنسان مسؤول ومحاسب عن كل مايتفوه به والمص...
9,RT @FAHAD_BN_KHALID: اخي الغالي رئيسنا الذهبي ...


In [ ]:
df_corpus = df_train.tweet_content.append(df_test.tweet_content).append(df_unlabeled.tweet_content)

In [ ]:
print(len(df_corpus))
df_corpus.head()

In [ ]:
'''
dframe = pd.DataFrame()
for i in range(1,15):
  path = "./drive/My Drive/Colab/NLP/Nadi Shared Task/Data/unlabeled_10M_"+str(i)+".tsv"
  t = pd.read_csv(path, delimiter='\t',header=0, names=['tweet_ID', 'tweet_content'])
  print("path = ",path,"\nlength = ",len(t))
  dframe = dframe.append(t)
print(len(dframe))
# dframe.tail()
#3,635,600
'''

'\ndframe = pd.DataFrame()\nfor i in range(1,15):\n  path = "./drive/My Drive/Colab/NLP/Nadi Shared Task/Data/unlabeled_10M_"+str(i)+".tsv"\n  t = pd.read_csv(path, delimiter=\'\t\',header=0, names=[\'tweet_ID\', \'tweet_content\'])\n  print("path = ",path,"\nlength = ",len(t))\n  dframe = dframe.append(t)\nprint(len(dframe))\n# dframe.tail()\n#3,635,600\n'

In [ ]:
'''
shutil.move("unlabeled_data_10M.tsv","./drive/My Drive/Colab/NLP/Nadi Shared Task/Data")
shutil.move("./NADI_release/dev_labeled.tsv","./drive/My Drive/Colab/NLP/Nadi Shared Task/Data")
shutil.move("./NADI_release/train_labeled.tsv","./drive/My Drive/Colab/NLP/Nadi Shared Task/Data")
'''

'\nshutil.move("unlabeled_data_10M.tsv","./drive/My Drive/Colab/NLP/Nadi Shared Task/Data")\nshutil.move("./NADI_release/dev_labeled.tsv","./drive/My Drive/Colab/NLP/Nadi Shared Task/Data")\nshutil.move("./NADI_release/train_labeled.tsv","./drive/My Drive/Colab/NLP/Nadi Shared Task/Data")\n'

# Data Preprocessing

In [ ]:
#From Moustafa Tohamy

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def remove_links(text):
  text = re.sub(r'http\S+', '', text)
  return text

def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def pre_process_document(text):
  text = remove_links(text)
  text = remove_punctuations(text)
  # text = remove_diacritics(text)
  text = remove_repeating_char(text)
  return text

pre_process_corpus = np.vectorize(pre_process_document)

# Utils

In [ ]:
def save_model(model,epoch,model_type):
  output_file = './saved_'+model_type+'_epoch'+str(epoch)+'.txt'
  print("Saving model to %s" % output_file)
  torch.save(model.state_dict(), output_file)
  shutil.move(output_file,models_path)

In [ ]:
def load_model(model,epoch,model_type):
  path = models_path + 'saved_'+model_type+'_epoch'+str(epoch)+'.txt'
  print("Loading model from %s" % path)
  model.load_state_dict(torch.load(path))
  return model
# model_embed_pretrained.load_state_dict(torch.load(path))


In [ ]:
def save_asafaya(model,models_path):
    print("Saving model to %s" % models_path)
    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(models_path)

In [ ]:
def show_model_parameters(model):
  # Get all of the model's parameters as a list of tuples.
  params = list(model.named_parameters())

  print('# of parameters = \n',len(params))
  print('==== Embedding Layer ====\n')
  for p in params[0:5]:
      print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

  print('\n==== First Transformer ====\n')
  for p in params[5:21]:
      print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

  print('\n==== Output Layer ====\n')
  for p in params[-4:]:
      print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
def prepare_data(dataset,train_size,batch_size):
  train_size = int(train_size * len(dataset)) #99% train_size
  val_size = len(dataset) - train_size #1% val size
  train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
  train_dataloader = DataLoader(train_dataset,sampler = RandomSampler(train_dataset),batch_size = batch_size)
  validation_dataloader = DataLoader(val_dataset,sampler = RandomSampler(val_dataset),batch_size = batch_size)
  print('training samples',len(train_dataset))
  print('validation samples',len(val_dataset))
  return train_dataloader,validation_dataloader

In [ ]:
def training_model(model,model_type,train_dataloader,validation_dataloader,epochs):
  training_loss = []
  validation_loss = []
  validation_acc = []
  optimizer = AdamW(model.parameters(),lr = 2e-5, eps = 1e-8 )
  total_steps = len(train_dataloader) * epochs
  warmup_steps = 0 
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = warmup_steps,num_training_steps = total_steps)
  start_time = time.time()
  for epoch in range(epochs):
      total_train_loss = 0
      model.train()
      print('Epoch: {}'.format(epoch+1))
      for step, batch in enumerate(Bar(train_dataloader)):
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)
          model.zero_grad()
          loss, logits = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask,labels=b_labels) #it computes loss function so it takes labels
          total_train_loss += loss.item()
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          optimizer.step()
          scheduler.step()

      avg_train_loss = total_train_loss / len(train_dataloader)  

      model.eval()
      save_model(model,epoch+1,model_type)
      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0
      print("Validation")
      for batch in validation_dataloader:
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)

          with torch.no_grad():        
            (loss, logits) = model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)
            total_eval_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()


      avg_val_loss = total_eval_loss / len(validation_dataloader)
      print("Validation Loss :", avg_val_loss )
      training_loss.append(avg_train_loss)
      validation_loss.append(avg_val_loss)

  elapsed_time = time.time() - start_time
  print("Done")
  print("Total training took :",str(datetime.timedelta(seconds=int(round((elapsed_time))))))
  return model,training_loss,validation_loss

# Transform

In [ ]:
class Transform_preprocess():
  def __call__(self,x):
    out = pre_process_corpus([x])[0]
    return out
class Transform_to_trigram():
  def __call__(self,x):
    out = " ".join(re.findall('...|..$|.$',x.replace(" ","")))
    return out
class Transform_to_bigram():
  def __call__(self,x):
    out = " ".join(re.findall('..|.$',x.replace(" ","")))
    return out

# Word Embedding Model

## Corpus Dataset

In [ ]:
class CorpusDataset(Dataset):
    def __init__(self,corpus,tokenizer,context_size = 2,preprocess=None,transform = None):
        self.tokens = []      
        self.dataset = []
        self.input_ids = []
        self.attention_masks = []
        self.output_ids = []
        self.preprocess = preprocess
        self.transform = transform
        self.tokenizer = tokenizer
        self.context_size = context_size
        print("Tokenizer Loading")
        self.tokenize_corpus(corpus)
        print("Context vector Loading")
        self.context_vector()
        print("Features Loading")
        self.get_inputs_features()
    def tokenize_corpus(self,corpus):   
        loading_length = len(corpus)
        print("Length = ",loading_length)
        for iter,sent in enumerate(corpus):
          if(self.preprocess):
            sent =self.preprocess(sent)
          if(self.transform):
            sent = self.transform(sent)
          self.tokens.append(sent.split())
          if(iter % 100000 == 0):
              loading = int(iter*100/loading_length)
              print("Loading Tokenizer:",loading,"%")

    def context_vector(self):
        loading_length = len(self.tokens)
        print("Length = ",loading_length)
        for iter,sentence in enumerate(self.tokens):
          for i in range(0,len(sentence)): #min len(sentence) = 2
              context = []
              for j in reversed(range(1,self.context_size+1)):
                  if((i-j) >= 0): #if sentence[i-j] != null
                      context.append(sentence[i-j])
              for j in range(1,self.context_size+1):
                  if((i+j) < len(sentence)): #if sentence[i+j] != null
                      context.append(sentence[i+j])
              target = sentence[i]

              if(len(context) == 0 or len(target.split()) == 0): 
                continue
              if(self.transform):
                self.dataset.append([self.transform("".join([input_word for input_word in context])),target])
              else:
                self.dataset.append([" ".join([input_word for input_word in context]),target])
          if(iter % 100000 == 0):
              loading = int(iter*100/loading_length)  
              print("Loading Context vector:",loading,"%")

    
    def show_dataset(self):
        for input_word,output_word in self.dataset:
            print(self.idx2word[input_word],self.idx2word[output_word])

    def get_inputs_features(self):
      loading_length = len(self.dataset)
      print("Length = ",loading_length)
      input_ids = []
      output_ids = []
      attention_masks = []
      for step,item in enumerate(self.dataset):
        sentences,label = item
        input_dict = self.tokenizer.encode_plus(
              sentences,                      # Sentence to encode.
              add_special_tokens = True, # Add '[CLS]' and '[SEP]'
              max_length = 16,           # Pad & truncate all sentences.
              pad_to_max_length = True,
              return_attention_mask = True,   # Construct attn. masks.
              return_tensors = 'pt',     # Return pytorch tensors.
          )
        output_dict = self.tokenizer.encode_plus(
              label,                      # Sentence to encode.
              add_special_tokens = False, # Add '[CLS]' and '[SEP]'
              max_length = 1,           # Pad & truncate all sentences.
              pad_to_max_length = False,
              return_attention_mask = False   # Construct attn. masks.
              # return_tensors = 'pt',     # Return pytorch tensors.
          )
        self.input_ids.append(input_dict['input_ids'])
        self.attention_masks.append(input_dict['attention_mask'])
        if(len(output_dict['input_ids']) == 0):
          output_dict['input_ids'].append(0)
        self.output_ids.append(torch.tensor(output_dict['input_ids']))
        if(step % 100000 == 0):
            loading = int(step*100/loading_length)
            print("Loading Features:",loading,"%")

      

    def __getitem__(self,idx):
      return self.input_ids[idx][0],self.attention_masks[idx][0],self.output_ids[idx][0]
            

    
    def __len__(self):
        return len(self.dataset)

## Bert Embdding Model

In [ ]:
class BertEmbedding(torch.nn.Module):
    """BERT model for classification.
    This module is composed of the BERT model with a linear layer on top of
    the pooled output.
    """
    def __init__(self,path="asafaya/bert-base-arabic" ):
        super(BertEmbedding, self).__init__()
        self.model_config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
          num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)
        self.bert = BertModel.from_pretrained(path)
        self.dropout = torch.nn.Dropout(self.model_config.hidden_dropout_prob)
        self.embedding2 = torch.nn.Linear(self.model_config.hidden_size, self.model_config.vocab_size_or_config_json_file)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _,pooled_output = self.bert(input_ids, token_type_ids, attention_mask)
        pooled_output = self.dropout(pooled_output)
        logits = self.embedding2(pooled_output)
        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            #Cross Entropy input.shape= [batch_size,num_labels] ,labels.shape = [batch_size]
            loss = loss_fct(logits,labels.view(-1))
        return loss,logits
    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True
        
    def save_pretrained(self,path):
        self.bert.save_pretrained(path)


In [ ]:
path_tokenizer = models_path+"tokenizer/"
if not os.path.exists(path_tokenizer):    
    os.makedirs(path_tokenizer)
    tokenizer = BertTokenizer.from_pretrained('asafaya/bert-base-arabic', do_lower_case=True)
    tokenizer.save_pretrained(path_tokenizer)
else:
    tokenizer = BertTokenizer.from_pretrained(path_tokenizer, do_lower_case=True)

In [ ]:
path_asafaya = models_path+"asafaya_arabic_model/"
if not os.path.exists(path_asafaya):    
    os.makedirs(path_asafaya)
    model_embed = BertEmbedding('asafaya/bert-base-arabic')
    model_embed.save_pretrained(path_asafaya)
else:
    model_embed = BertEmbedding(path_asafaya)
    
# model_embed.to(device)

In [ ]:
# load model
embed_epoch = 1
model_embed = load_model(model_embed,embed_epoch,'embed')
model_embed.to(device)

In [ ]:
# corpus = pre_process_corpus(df_corpus.values)
dataset_cbow = CorpusDataset(df_corpus.values,tokenizer,context_size = 10,preprocess=Transform_preprocess(),transform = None)
print("Dataset training samples :",len(dataset_cbow))
print(dataset_cbow[0])

In [ ]:
del df_unlabeled
del df_corpus
gc.collect()

In [ ]:
train_dataloader ,validation_dataloader = prepare_data(dataset_cbow,0.99,64) #dataset_cbow 

In [ ]:
model_embed,training_loss_embed,validation_loss_embed  = training_model(model_embed,"embed",train_dataloader,validation_dataloader,epochs=10)

# Classification Model

## Nadi Dataset

In [ ]:
class dataset_nadi():
  def __init__(self,sentences,labels,labels_vocab,tokenizer,preprocess=None,transform=None):
    self.preprocess = preprocess
    self.transform=transform
    self.sentences = sentences
    self.labels = labels
    self.tokenizer = tokenizer
    self.labels_vocab = labels_vocab
    self.label2idx = self.label2idx()
    self.idx2label = self.idx2label()
    self.labels_indices = [self.label2idx[label] for label in self.labels]
    self.max_length,self.max_len_word = self.get_max_length()
    self.inputs_id,self.attention_masks = self.get_inputs_features()

  def label2idx(self):
    return {w:i for i,w in enumerate(self.labels_vocab)}

  def idx2label(self):
    return {i:w for i,w in enumerate(self.labels_vocab)}

  def get_max_length(self):
    max_len = 0.
    max_len_idx = 0
    for i,sent in enumerate(self.sentences):
        input_ids = self.tokenizer.encode(sent, add_special_tokens=True)
        if(len(input_ids) > max_len):
          max_len = len(input_ids)
          max_len_idx = i
    return max_len,self.sentences[max_len_idx]

  def get_inputs_features(self):
    input_ids = []
    attention_masks = []
    for sent in self.sentences:
      if(self.preprocess):
        sent =self.preprocess(sent)
      if(self.transform):
        sent = self.transform(sent)
      encoded_dict = self.tokenizer.encode_plus(
            sent,                      # Sentence to encode.
            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
            max_length = 200,           # Pad & truncate all sentences.
            pad_to_max_length = True,
            return_attention_mask = True,   # Construct attn. masks.
            return_tensors = 'pt',     # Return pytorch tensors.
        )
      input_ids.append(encoded_dict['input_ids'])
      attention_masks.append(encoded_dict['attention_mask'])
    return input_ids,attention_masks

  # def get_bert_input_format(self):
      

  def __getitem__(self,idx):
      input_ids = self.inputs_id[idx][0]
      attention_masks = self.attention_masks[idx][0]
      labels_ids = torch.tensor(self.labels_indices[idx])
      return input_ids,attention_masks,labels_ids

  def __len__(self):
      return len(self.sentences)

## Bert Nadi Model

In [ ]:
class Bert_Nadi_Task(torch.nn.Module):
    """BERT model for classification.
    This module is composed of the BERT model with a linear layer on top of
    the pooled output.
    """
    def __init__(self,Model=None,num_labels =  21):
        super(Bert_Nadi_Task, self).__init__()
        self.num_labels = num_labels
        model_config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
          num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)
        self.bert = Model
        self.norm = torch.nn.LayerNorm((model_config.vocab_size_or_config_json_file,), eps=model_config.layer_norm_eps, elementwise_affine=True)
        self.tanh = torch.nn.Tanh()
        self.dropout = torch.nn.Dropout(model_config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(model_config.vocab_size_or_config_json_file, num_labels)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _,logits = self.bert(input_ids, token_type_ids, attention_mask)
        norm_output = self.norm(logits)
        out = self.dropout(self.tanh(norm_output))
        logits = self.classifier(out)
        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            #Cross Entropy input.shape= [batch_size,num_labels] ,labels.shape = [batch_size]
            loss = loss_fct(logits,labels.view(-1))
        #test
        return loss,logits

    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True


In [ ]:
#load embedding model and start nadi model from scratch
epoch_embed = 1
model_nadi = Bert_Nadi_Task(load_model(model_embed,epoch_embed,"embed"))
epoch_nadi = 4
model_nadi = load_model(model_nadi,epoch_nadi,"nadi")
model_nadi.to(device)

Loading model from Models/saved_embed_epoch1.txt
Loading model from Models/saved_nadi_epoch4.txt


Bert_Nadi_Task(
  (bert): BertEmbedding(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(32000, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
             

### Prepare data

In [ ]:
labels = df_train.country_label.values
#this is ordered unique list however it'll be changed if any row appended in dataset so use it as static if it doesn't match next line
#labels_vocab = ['Iraq', 'Egypt', 'Morocco', 'Libya', 'United_Arab_Emirates', 'Mauritania', 'Saudi_Arabia', 'Bahrain', 'Syria', 'Djibouti', 'Lebanon', 'Oman', 'Palestine', 'Algeria', 'Somalia', 'Jordan', 'Tunisia', 'Kuwait', 'Yemen', 'Sudan', 'Qatar']

In [ ]:
nadi_dataset = dataset_nadi(df_train.tweet_content.values,labels,labels_vocab,tokenizer,preprocess=Transform_preprocess(),transform=None)
train_dataloader ,validation_dataloader = prepare_data(nadi_dataset,0.95,8)

training samples 19950
validation samples 1050


### Training Model

In [ ]:
model_nadi,train_loss_nadi,valid_loss_nadi = training_model(model_nadi,'nadi',train_dataloader,validation_dataloader,epochs=4)

Epoch: 1
19950/19950: [===============================>] - ETA 1.7ssss
Saving model to ./saved_nadi_epoch1.txt
Validation
Validation Loss : 2.121380985234723
Epoch: 2
19950/19950: [===============================>] - ETA 0.3sss
Saving model to ./saved_nadi_epoch2.txt
Validation
Validation Loss : 2.1113187239477127
Epoch: 3
19950/19950: [===============================>] - ETA 0.3sss
Saving model to ./saved_nadi_epoch3.txt
Validation
Validation Loss : 2.2292832845088206
Epoch: 4
19950/19950: [===============================>] - ETA 0.3sss
Saving model to ./saved_nadi_epoch4.txt
Validation
Validation Loss : 2.587091742139874
Done
Total training took : 0:57:24


# Testing Model

## Utils

In [ ]:
def predict_model(model,prediction_dataloader,device):
  model.eval()
  predictions , true_labels = [], []
  for batch in prediction_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[1] 
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    predictions.append(logits)
    true_labels.append(label_ids)
  flat_predictions = np.concatenate(predictions, axis=0)
  flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
  flat_true_labels = np.concatenate(true_labels, axis=0)
  print('DONE.')
  return flat_predictions,flat_true_labels

In [ ]:
def show_predictions(test_dataset,predictions_items,labels_items,sentences,samples = 10):
  true_predicted = 0 
  for i in range(samples):
    print("Sentence :",sentences[i])
    print("True label :",test_dataset.idx2label[labels_items[i]])
    print("predicted Label :" , test_dataset.idx2label[predictions_items[i]])
    print("\n")
    if(predictions_items[i] == labels_items[i]):
      true_predicted += 1
  print("with number of samples =",samples," true predicted = ",true_predicted, " acc = ",(true_predicted*100.0/samples))

In [ ]:
def show_score(epochs,flat_true_labels,flat_predictions):
  #calculate accuracy for test data
  print("for number of epochs = ",epochs)
  print("Testing Accuracy using accuracy score=",accuracy_score(flat_true_labels, flat_predictions))
  print("Testing Accuracy using macro recall score=",recall_score(flat_true_labels, flat_predictions, average='macro')) #average='micro' same for accuracy_score
  print("Testing Accuracy using macro precision score=",precision_score(flat_true_labels, flat_predictions ,average='macro')) #average='micro' same for accuracy_score
  print("Testing Accuracy using macro f1 avg =",f1_score(flat_true_labels, flat_predictions, average='macro')) #average='micro' same for accuracy_score

## Test Score

In [ ]:
sentences_test = df_test.tweet_content.values
labels_test = df_test.country_label.values
test_dataset = dataset_nadi(sentences_test,labels_test,labels_vocab,tokenizer,preprocess=Transform_preprocess())
test_dataloader = DataLoader(test_dataset,sampler = RandomSampler(test_dataset),batch_size = 32)

In [ ]:
flat_predictions,flat_true_labels = predict_model(model_nadi,test_dataloader,device)
show_score(4,flat_true_labels,flat_predictions)

DONE.
for number of epochs =  4
Testing Accuracy using accuracy score= 0.40205769618721
Testing Accuracy using macro recall score= 0.23793407709999
Testing Accuracy using macro precision score= 0.2547309267575428
Testing Accuracy using macro f1 avg = 0.24051259820371235


In [ ]:
show_predictions(test_dataset,flat_predictions,flat_true_labels,sentences_test,samples = 10)

Sentence : @Osama__zahrani ايسكو لاعب اليوم :) اسيست وهدف
True label : Libya
predicted Label : Djibouti


Sentence : بعد صلاه الفجر بقا
True label : Algeria
predicted Label : Algeria


Sentence : إن شاء الله هذه المرة يكون من نصيبي
True label : Algeria
predicted Label : Morocco


Sentence : ههههههههههههههههه خلي السوداني يزغبك
True label : Saudi_Arabia
predicted Label : Iraq


Sentence : كل حاجة محسوبة يا جماعة والله
True label : Qatar
predicted Label : Saudi_Arabia


Sentence : @kasimf اللهم ضيق عليه أنفاسه وعسر ميتته إلى أجل هو بالغه..
True label : Morocco
predicted Label : Morocco


Sentence : يعني ما يحتاج نلبس ثقيل
True label : Iraq
predicted Label : Oman


Sentence : (ولئن سألتهم من خلقهم ليقولن الله فأنى يؤفكون) [الزخرف:87]
True label : Morocco
predicted Label : Tunisia


Sentence : آستودعك يا ربي شخصاً ينبض في قلبي كل مره ، ولا يفارق آفكاري
True label : United_Arab_Emirates
predicted Label : Egypt


Sentence : قُل لَّن يُصِيبَنَا إِلَّا مَا كَتَبَ اللَّهُ لَنَا هُوَ مَوْلَانَا 

## Predict Score

In [ ]:
sentences__ = ['اسكندريه','معتز']
labels__ = ['Egypt','Oman']
dataset__ = dataset_nadi(sentences__,labels__,labels_vocab,tokenizer)
dataloader__ = DataLoader(dataset__,sampler = RandomSampler(dataset__),batch_size = 1)

In [ ]:
flat_predictions__,flat_true_labels__ = predict_model(model_nadi,dataloader__,device)
show_predictions(dataset__,flat_predictions__,flat_true_labels__,sentences__,samples = len(sentences__))
# show_score(4,flat_true_labels__,flat_predictions__)

DONE.
Sentence : اسكندريه
True label : Egypt
predicted Label : Egypt


Sentence : معتز
True label : Oman
predicted Label : Libya


with number of samples = 2  true predicted =  1  acc =  50.0
